In [2]:
# LIBS IMPORT
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [13]:
# READ DATASET
df = pd.read_csv("../../dataset/data_vivareal_21_03_2022_18_38_14.csv").drop(0)

In [14]:
df.head()

,id,url,address,area,rooms,batrooms,garage,amenities,value,condominium,date
1,2553339688,https://www.vivareal.com.br/imovel/apartamento...,"Rua Monsenhor Manoel Vicente, 597 - Água Verde...",51 m²,2 Quartos,1 Banheiro,1 Vaga,Aceita animais\nCozinha americana\nMobiliado\n...,R$ 325.000,Condomínio: R$ 350,2022-03-21 18:38:38
2,2074895474,https://www.vivareal.com.br/imoveis-lancamento...,"Rua Professor Guido Straube, 775 - Vila Izabel...",72-78 m²,3 Quartos,2 Banheiros,2 Vagas,NaN,R$ 598.000,NaN,2022-03-21 18:38:38
3,2540019474,https://www.vivareal.com.br/imovel/apartamento...,"Rua Monsenhor Manoel Vicente, 316 - Água Verde...",94 m²,3 Quartos,2 Banheiros,2 Vagas,Mobiliado\nChurrasqueira\nElevador\nCondomínio...,R$ 680.000,Condomínio: R$ 530,2022-03-21 18:38:38
4,2518442439,https://www.vivareal.com.br/imovel/apartamento...,"Rua Coronel Dulcídio, 1756 - Água Verde, Curit...",54 m²,2 Quartos,1 Banheiro,-- Vaga,Mobiliado\nPortão eletrônico\nAceita animais\n...,R$ 215.000,Condomínio: R$ 345,2022-03-21 18:38:39
5,2477728884,https://www.vivareal.com.br/imovel/apartamento...,"Avenida Iguaçu, 2960 - Água Verde, Curitiba - PR",102 m²,3 Quartos,3 Banheiros,2 Vagas,Piscina\nMobiliado\nChurrasqueira\nElevador\nC...,R$ 650.000,Condomínio: R$ 1.000,2022-03-21 18:38:39


In [57]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 36617 entries, 1 to 36617
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   id           36617 non-null  object
 1   url          36617 non-null  object
 2   address      36617 non-null  object
 3   area         36617 non-null  object
 4   rooms        36617 non-null  object
 5   batrooms     36617 non-null  object
 6   garage       36617 non-null  object
 7   amenities    26952 non-null  object
 8   value        36617 non-null  object
 9   condominium  16795 non-null  object
 10  date         36617 non-null  object
 11  area_clean   36617 non-null  object
dtypes: object(12)
memory usage: 4.6+ MB


# Cleaning data

In [30]:
# The area column have the m^2 string, remove it and let only the number
# df["area_clean"] = [x.split(" ")[0] for x in df["area"]]
df["area"] = df["area"].apply(lambda x : x.split(" ")[0])

In [50]:
# df.loc["-" in df["area"], "area"]

In [55]:
df.loc[df["area"].str.contains('-'), "url"].count()

3185